In [3]:
import numpy as np
import random as rand
#Random used in base generation for Shores Algorithm

## Handling ASCII
Functions convert between a string such as 'this is a string' and a list of corresponding ASCII integers
as in [116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 115, 116, 114, 105, 110, 103]

In [5]:
def conv_to_ASCII(string):
    ascii_values = [ord(char) for char in string]
    return (ascii_values)

In [6]:
def ASCII_to_string(ascii_list):
    return ''.join(chr(num) for num in ascii_list)

Example Usage

In [8]:
conv_to_ASCII('this is a string')

[116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 115, 116, 114, 105, 110, 103]

In [9]:
ASCII_to_string([116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 115, 116, 114, 105, 110, 103])

'this is a string'

# Binary Conversion
Converts a binary string to an integer.

In [11]:
def convert_binary_to_int(bin_str):
    val = 0
    for char in bin_str:
        val *= 2
        val += int(char) 
    return val

# Modular Expoinentiation and Inversion
First function implements algorithm for fast modular exponentiation. **Algorithm 2.6**\
Second function implements the extended euclidean algorithm.

In [13]:
def mod_exponentiate(base,power,mod):
    true_base=base%mod
    res=true_base
    
    for bit in bin(power)[3:]:
        res=(res*res)%mod
    
        if int(bit)==1:
            res=(true_base*res)%mod
    return res

In [14]:
def modular_inverse(a, m):
    original_m, x, y = m, 1, 0
    a1, b1 = a, m

    while b1:
        q = a1 // b1
        a1, b1 = b1, a1 % b1
        x, y = y, x - q * y

    if a1 != 1:
        raise ValueError(f"No modular inverse: {a} and {m} are not coprime.")

    return x % original_m  # Ensure positive inverse

# Chinese Remainder Theorem
Implements Garner's CRT algorithm **Algorithm 2.8**\
'prod' is a sub function in 'fast_crt' for multiplying all elements in a list.

In [16]:
def prod(lst):
    """Returns the product of elements in a list."""
    result = 1
    for num in lst:
        result *= num
    return result

In [17]:
def fast_crt(v,m):
    t = len(m)
    C = [1] * t
    
    # Step 1: Compute C values
    for i in range(1, t):
        C[i] = 1
        for j in range(i):
            u = modular_inverse(m[j],m[i])
            C[i] = (C[i] * u) % m[i]
    
    # Step 2: Compute x
    x = v[0]
    for i in range(1, t):
        u = ((v[i] - x) * C[i]) % m[i]
        x += u * (prod(m[:i]))
    
    return x

# Encryption
Takes a message as a *string* and uses fast modular exponentiation as in **Algorithm 2.6**.\
Outputs a list of cyphertext message units (in integer form).

In [19]:
def fast_encrypt(Message, PublicExponent, Modulus): 
    encrypted_lst = [mod_exponentiate(code,PublicExponent,Modulus) for code in conv_to_ASCII(Message)]
    return encrypted_lst

# Decryption
'naive_decrypt' takes as input as *list of integers* in ASCII (or some other format) and uses fast modular exponentiation as in **Algorithm 2.6** for decryption.\
'fast_decrypt' takes as input a *list of integers* in ASCII (or some other format) and uses CRT modular exponentiation for decryption. Note that this requires $p$ and $q$ by as a part of the private key.

In [21]:
def fast_decrypt(cyphertext,p,q,d):
    decrypted_lst=[fast_crt([mod_exponentiate(c_message_unit,d%(p-1),p),mod_exponentiate(c_message_unit,d%(q-1),q)],[p,q]) for c_message_unit in cyphertext]
    return decrypted_lst

In [22]:
def naive_decrypt(input, PublicExponent, Modulus):
    encrypted_lst = [mod_exponentiate(code,PublicExponent,Modulus) for code in input]
    return encrypted_lst

# Factorisation
Implements a classical version of Shor's Factorisation Algorithm. This differs slightly from **Algorithm 2.4** in that we allow for odd order by using a scaling gcd function.\
'scaled_gcd' is a sub function that allows for finding the gcd of numbers of the form $z+\frac{1}{2}:z\in\mathbb{Z}$.\
'find_order' determines the order of $a$ in $\mathbb{Z}/N\mathbb{Z}$.

In [24]:
def scaled_gcd(a,b):
    scale = 2 if (a % 1 == 0.5 or b % 1 == 0.5) else 1
    a, b = int(a * scale), int(b * scale)

    return (np.gcd(a,b)/scale)

In [25]:
def find_order(a, N):
    for r in range(1, N):
        if mod_exponentiate(a, r, N) == 1:
            return r
    return None  # Should never happen for valid choices of a

In [26]:
def shores_factorise(N):
    if N%2==0:
        return 2

    while True:
        base_test=rand.randint(2,N-1)
        base=np.gcd(base_test,N)
        
        if N>base>1:
            return base,N//base
            
            
        else:
            r=find_order(base,N) 
            
            factor1=int(scaled_gcd(N,(base**(r/2)-1)))
            factor2=int(scaled_gcd(N,(base**(r/2)+1)))

            if N>factor1>1:
                return factor1,N//factor1
            if N>factor2>1:
                return factor2,N//factor2